<a href="https://colab.research.google.com/github/dongjaeseo/colab/blob/main/adult_child_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd

import os

# 트레인/테스트 데이터 이름을 프린트 하는부분 : 모델의 학습에는 영향이 없는부분
for dirname, _, filenames in os.walk('/content/drive/MyDrive/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# 필요한 라이브러리 임포트
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Input
from keras.regularizers import l2

In [ ]:
# 트레인 데이터 불러오기 
'''
트레인 데이터는 어른과 아이사진 각각 340개씩 총 680장이 들어있다
이 중 트레인 데이터를 0.8 : 0.2 비율로 나눠
0.8 은 모델의 학습에 사용되는 트레인데이터
0.2 는 모델의 학습도중 모델이 맞는 방향으로 학습하는지 판단하기 위한 밸리데이션 데이터로 나눠진다

0.8 비율을 가진 트레인 데이터는 총 544 장이고
0.2 비율을 가진 밸리데이션 데이터는 총 136 장이다

seed 는 매번 모델을 학습시킬때마다 같은 결과를 얻기 위한 난수생성 시드를 의미한다

batch_size 는 모델을 학습 시에 한 배치 단위로 훈련을 하는데
이때 한 배치에 들어가는 이미지의 수 = 32 장
>> 1 배치에는 32장의 이미지가 들어있다
'''
train = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/kaggle/train',
                                                            validation_split=0.2,
                                                            batch_size=32,
                                                            image_size=(128,128),
                                                            subset="training",
                                                            seed=123)

validation = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/kaggle/train',
                                                            validation_split=0.2,
                                                            batch_size=32,
                                                            image_size=(128,128),
                                                            subset="validation",
                                                            seed=123)

In [ ]:
# 테스트 데이터 불러오기
'''
테스트 데이터는 학습된 모델의 성능을 평가하기 위한 데이터셋이다
모델의 학습에는 관여하지 않고 성능만을 평가하기 위함

테스트 데이터는 아이와 어른 각 60 장의 사진으로 총 120장의 이미지 데이터가 있다
'''
test = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/kaggle/test',
                                                            batch_size=32,
                                                            image_size=(128,128),
                                                            seed=123)

In [ ]:
# class_names 라는 변수에 / 모델의 아웃풋인 / 어른과 아이라는 라벨값을 붙여줄것이다
class_names = train.class_names
print(class_names) # ['adults', 'children']

In [ ]:
# 이 중 9장의 사진을 출력해본 결과
# 사진에 맞게 라벨(어른/아이)이 매칭 되어있다
# 이때 사진은 모델의 인풋이 되고 라벨은 모델의 아웃풋이 된다
# >>> 목표는 모델에 사진데이터를 넣었을때 어른인지 아이인지 판별하는 것

plt.figure(figsize=(10,10))
for images, labels in train.take(1):
    
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
# 모델의 구조
# 모델은 레이어 여러장으로 구성되어있다

# Sequential - 모델의 여러 레이어를 하나로 묶어 한개의 모델로 만들어준다
# Rescaling - 이미지 데이터는 기존 0~255 픽셀값으로 이뤄져있다 / 이를 0~1 값으로 재조정해주어 모델의 학습에 용이해지게 만들어준다
# Conv2D - 컨볼루션 레이어라고도 한다 - 이미지의 특성을 추출하기 위한 레이어이고 / filter 는 다음 차원의 크기 (128, 128, 3) 이 필터 16 컨볼루션 레이어를 거치면 (128, 128, 16) 이 된다
# kernel_size = 커널사이즈, 컨볼루션 레이어에서 연산을 하기 위한 파라미터로 3X3 커널을 사용하였다 / padding - 이미지의 크기를 똑같이 (128, 128) 로 맞춰주기 위한 파라미터
# kernel_regularizer - 모델의 오버피팅 (트레인 데이터에만 맞춰지는 과도한 학습) 을 막기위한 파라미터

# Activation - 모델의 비선형성을 도입하기 위한 활성화 함수, relu 를 사용하였다 / relu(x) = max(0, x)
# MaxPooling - 이미지 사이즈 축소
# BatchNormalization - 모델의 오버피팅을 막기위한 파라미터

# Flatten - 이미지 데이터를 Dense 레이어에 통과시키기 위해 3차원 데이터를 1차원으로 바꿔준다
# Dense - 1차원 데이터 -> 1차원 데이터를 통과시키는 레이어

# 마지막 덴스 레이어는 활성화 함수가 sigmoid 이다 - 어른과 아이로 구분하는 이진분류 문제이기 때문

model = Sequential([
    
    tf.keras.layers.Rescaling(1/.255, input_shape=(128, 128, 3)),
    
    tf.keras.layers.Conv2D(filters=16,kernel_size= (3,3),padding='same',kernel_regularizer=l2(7e-4)),
    tf.keras.layers.BatchNormalization(trainable=False),
    tf.keras.layers.Activation("relu"), 
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same'),
    
    tf.keras.layers.Conv2D(filters=32,kernel_size= (3,3),padding='same',kernel_regularizer=l2(7e-4)),
    tf.keras.layers.BatchNormalization(trainable=False),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same'),
    
    tf.keras.layers.Conv2D(filters=32,kernel_size= (3,3),padding='same',kernel_regularizer=l2(7e-4)),
    tf.keras.layers.BatchNormalization(trainable=False),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=16,kernel_regularizer=l2(5e-4)),
    tf.keras.layers.BatchNormalization(trainable=False),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(units=32,kernel_regularizer=l2(5e-4)),
    tf.keras.layers.BatchNormalization(trainable=False),
    tf.keras.layers.Activation("relu"),
    
    tf.keras.layers.Dense(units=1,activation='sigmoid'),    
])

In [ ]:
# 모델의 구조를 보여주는 코드
# 모델 학습에는 영향이 없다
model.summary()

In [ ]:
# optimizer - 모델을 학습할때 오차를 줄이기 위한 과정을 정한다
# 아담 옵티마이저를 사용하였고, learning_rate 는 학습율을 말한다 / 높으면 오버피팅의 위험이 있고 / 너무 낮으면 학습이 더디다는 단점이 있다
# beta, epsilon, amsgrad 는 아담 옵티마이저의 파라미터
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.98,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam'
)

# 미리 정의해둔 모델 구조에 학습을 위한 파라미터들인 옵티마이저, 오차함수를 넣어준다
# 오차함수는 binarycrossentropy 를 사용하였는데, 이진분류에서 쓰이는 오차함수이다
# metrics 는 모델의 훈련중, 그리고 훈련 후 보여주는 평가지표이다. / 정확도 / 오차함수 를 사용

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['Accuracy',tf.keras.metrics.BinaryCrossentropy()])

epochs = 40

history = model.fit(train,epochs=epochs,validation_data=test)

에폭이 증가함에 따라 트레인 정확도는 1에 가까워 지지만 밸리데이션 정확도는 0.6 정도에서 머무는것을 확인할수 있다

모델이 학습을 아예 못했다면 단순히 어른/아이를 무분별하게 분류할테니 조금은 학습이 된걸 확인할수있다.

In [ ]:
# 모델의 학습과정중 정확도의 변화를 에폭의 진행과 함께 플롯한 그래프
# X축은 에폭 Y축은 정확도 - 여기서 정확도는 모델이 정확하게 아이 혹은 어른을 분류한 결과

# 모델의 에폭에 따라
# 모델의 학습이 정상적이라면 트레인 정확도는 항상 올라간다
# 이에 맞게 밸리데이션 정확도가 같이 상승한다면 학습이 잘되고 있다는 뜻

# 하지만 트레인 정확도가 올라감과 함께 밸리데이션 정확도는 떨어지고 있다면
# 주어진 이미지에 대해선 분류를 잘하지만 / 본적 없는 이미지에 대한 분류는 잘 못하고 있다는 뜻
# >> 오버피팅을 의심할수 있다

plt.figure(figsize=(10,8))
plt.plot(history.history['Accuracy'],'r',label='train accuracy')
plt.plot(history.history['val_Accuracy'],'b',label='test accuracy')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Graph')
plt.legend();

밸리데이션 정확도가 시간에 따라 점차 증가하는것을 주목

(극초반은 아직 학습이 덜된 상태에서 막 찍는것)

In [ ]:
# 트레인 학습중 기록된 평가지표들의 종류
# 학습에는 영향이 없다
history.history.keys()

In [ ]:
# 이번엔 오차를 그래프로 그려보았을때
# X축은 에폭 Y축은 오차
# 인공신경망 모델은 주로 오차를 줄이는 방향으로 학습이 되기에
# 트레인 및 밸리데이션 오차가 둘다 줄어드는 방향이 베스트

# 트레인 정확도와 비슷하게
# 트레인 오차가 내려감과 함께 밸리데이션 오차는 올라가고 있다면
# 주어진 이미지에 대해선 분류를 잘하지만 / 본적 없는 이미지에 대한 분류는 잘 못하고 있다는 뜻
# >> 오버피팅을 의심할수 있다

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
!pip install nbconvert
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic
!jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/adult_child_classification.ipynb"